### Jul AdventKalender D9

https://adventofcode.com/2022/day/9

In [1]:
import pandas as pd
import numpy as np

#### Day 9.1  

There is a bridge which one can simulate its stability by measuring the movements of the rope (where H is the Head of the rope, T is the Tail). Given a set  of movements of H (in the data file). It looks like for example (where R, U, L, D means moving right, up, left, down, the number means steps. So R 4 means H moves 4 steps to the right):

    R 4
    U 4
    L 3
    D 1
    R 4
    D 1
    L 5
    R 2

There is also a rule for how T moves with H:

* If the head is ever two steps **directly** up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:
        .....    .....    .....
        .TH.. -> .T.H. -> ..TH.
        .....    .....    .....

* Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:
        .....    .....    .....
        .....    ..H..    ..H..
        ..H.. -> ..... -> ..T..
        .T...    .T...    .....
        .....    .....    .....

By following the movements of H according to the data file, and executing and movements of T following the above rules, one can simulate how the rope moves. Specifically, one wants to count up **all of the positions the tail visited at least once** (including the start position s). For example, if the initial state is 

    == Initial State ==

    ......
    ......
    ......
    ......
    H.....  (H covers T, s)

The above example of movements of H, the positions that T has visited is:

    ..##..
    ...##.
    .####.
    ....#.
    s###..

which gives a total number of positions of 13.

Calculate how many positions does the tail of the rope visit at least once according to the given full data file.

In [2]:
data_moves = pd.read_table('data/input9.txt', sep=" ", header=None).to_numpy()
data_moves

array([['L', 1],
       ['D', 2],
       ['R', 2],
       ...,
       ['U', 2],
       ['R', 11],
       ['U', 5]], dtype=object)

In [3]:
def _checkEdges(pos_to):
    global nrow, ncol
    #pos_to[0] = max(0, min(pos_to[0], nrow - 1))
    #pos_to[1] = max(0, min(pos_to[1], ncol - 1))
    if pos_to[0] < 0 or pos_to[0] > nrow - 1 or pos_to[1] < 0 or pos_to[1] > ncol - 1:
        print('Map too small!',pos_to)
    return pos_to
def _markPosT(pos_T):
    global state
    state[pos_T[0],pos_T[1]] = 1
def _isTouching(pos_H, pos_T):
    return pos_T == pos_H or (abs(pos_T[0]-pos_H[0]) <= 1 and abs(pos_T[1]-pos_H[1]) <= 1)
def moveH(pos_from, direction):
    if direction in ['L','R']:
        pos_to = [pos_from[0], pos_from[1]-1 if direction == 'L' else pos_from[1]+1]
    else: # ['U','D']
        pos_to = [pos_from[0]-1 if direction == 'U' else pos_from[0]+1, pos_from[1]]
    pos_to = _checkEdges(pos_to)
    return pos_to
def moveT(pos_from, pos_H, if_track=True):
    if _isTouching(pos_from, pos_H):
        pos_to = pos_from
    else:
        if pos_H[1] != pos_from[1] and pos_H[0] != pos_from[0]: # move diagonally
            pos_to = [pos_from[0], pos_from[1]+1] if pos_H[1] > pos_from[1] else [pos_from[0], pos_from[1]-1]
            pos_to = [pos_to[0]+1, pos_to[1]] if pos_H[0] > pos_to[0] else [pos_to[0]-1, pos_to[1]]
            pos_to = _checkEdges(pos_to)
            if if_track: _markPosT(pos_to)
        elif pos_H[1] != pos_from[1]: # move directly up/down
            pos_to = [pos_from[0], pos_from[1]+1] if pos_H[1] > pos_from[1] else [pos_from[0], pos_from[1]-1]
            pos_to = _checkEdges(pos_to)
            if if_track: _markPosT(pos_to)
        else: # move directly left/right
            pos_to = [pos_from[0]+1, pos_from[1]] if pos_H[0] > pos_from[0] else [pos_from[0]-1, pos_from[1]]
            pos_to = _checkEdges(pos_to)
            if if_track: _markPosT(pos_to)
    return pos_to

In [4]:
nrow = 600
ncol = 600
pos_s = [300,300] # start position
pos_H = pos_s
pos_T = pos_s
state = np.zeros((nrow, ncol), dtype=int)
state[pos_T[0],pos_T[1]] = 1 # T has visited
state

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
for move_H in data_moves:
    for i in range(move_H[1]):
        pos_H = moveH(pos_H, move_H[0])
        pos_T = moveT(pos_T, pos_H)
print('Current position of H', pos_H)
print('Current position of T', pos_T)
print('All positions of T has visited', sum(sum(state)))

Current position of H [532, 506]
Current position of T [533, 506]
All positions of T has visited 5619


#### Day 9.2

Rather than two knots (H and T), one now must simulate a rope consisting of **10** knots. One knot is still the head of the rope and moves according to the series of motions (the data file). Each knot further down the rope follows the knot in front of it using the same rules as before. And now, only the positions visited by the **last knot (9)** need to be tracked.

For a second example:

    R 5
    U 8
    L 8
    D 3
    R 17
    D 10
    L 25
    U 20
        
The state after the movements now become:

    ..........................
    ..........................
    ..........................
    ..........................
    ..........................
    ..........................
    ..........................
    ..........................
    ..........................
    #.........................
    #.............###.........
    #............#...#........
    .#..........#.....#.......
    ..#..........#.....#......
    ...#........#.......#.....
    ....#......s.........#....
    .....#..............#.....
    ......#............#......
    .......#..........#.......
    ........#........#........
    .........########.........
    
Now, the tail (9) visits 36 positions (including s) at least once.

In [8]:
nrow = 600
ncol = 600
pos_s = [300,300] # start position
pos_H = pos_s
pos_Ts=[]
for i in range(9):
    pos_Ts.append(pos_s)
state = np.zeros((nrow, ncol), dtype=int)
state[pos_Ts[0][0],pos_Ts[0][1]] = 1 # any T has visited

In [9]:
for move_H in data_moves:
    for i in range(move_H[1]):
        pos_H = moveH(pos_H, move_H[0])
        for i in range(9):
            pos_from = pos_H if i == 0 else pos_Ts[i-1]
            iftrack = i == 8
            pos_Ts[i] = moveT(pos_Ts[i], pos_from, iftrack)
print('Current position of H', pos_H)
print('Current position of T', pos_Ts)
print('All positions of T has visited', sum(sum(state)))

Current position of H [532, 506]
Current position of T [[533, 506], [534, 506], [535, 505], [535, 504], [535, 503], [536, 502], [537, 502], [538, 502], [538, 503]]
All positions of T has visited 2376
